In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import re

books = pd.read_csv('/opt/ml/data/books.csv')
users = pd.read_csv('/opt/ml/data/users.csv')
ratings = pd.read_csv('/opt/ml/data/train_ratings.csv')

In [6]:
books.loc[books['category'].notnull()]['category'].value_counts()

category
['Fiction']                                               32956
['Juvenile Fiction']                                       5804
['Biography & Autobiography']                              3320
['History']                                                1925
['Religion']                                               1818
                                                          ...  
['Disguise']                                                  1
['Country lawyers']                                           1
['Humorous stories, Brazilian']                               1
['Coasts']                                                    1
['Authors, Canadian (English) 20th century Biography']        1
Name: count, Length: 4292, dtype: int64

In [4]:
books.isna().sum()/len(books)

isbn                   0.000000
book_title             0.000000
book_author            0.000007
year_of_publication    0.000000
publisher              0.000000
img_url                0.000000
language               0.449468
category               0.460326
summary                0.449468
img_path               0.000000
dtype: float64

In [5]:
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])
publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values
for publisher in modify_list:
    try:
        number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
        right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
        books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
    except: 
        pass
books['publisher'].nunique()

1523

In [6]:
books['category'] = books['category'].str.lower()
categories = ['garden','crafts','physics','adventure','music','fiction','nonfiction','science','science fiction','social','homicide',
 'sociology','disease','religion','christian','philosophy','psycholog','mathemat','agricult','environmental',
 'business','poetry','drama','literary','travel','motion picture','children','cook','literature','electronic',
 'humor','animal','bird','photograph','computer','house','ecology','family','architect','camp','criminal','language','india']

for category in categories:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category'] = category

In [7]:
books.isna().sum()
count_category_df = pd.DataFrame(books['category'].value_counts())
count_category_df[:20]

,count
category,
fiction,41264
biography autobiography,3326
science,2308
history,1927
religion,1826
humor,1289
business,1147
cook,1125
body mind spirit,1113


In [8]:
count_category_df[20:]

,count
category,
children,516
philosophy,491
nature,490
reference,471
performing arts,457
...,...
oyster industry,1
african american artists,1
sarek fictitious character,1


In [9]:
train = books[books['category'].notnull()]
train

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,Perennial,http://images.amazon.com/images/P/0060973129.0...,en,1940 1949,"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,medical,"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,fiction,A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,history,"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149562,0670528951,Orson Welles,Barbara Leaming,1985.0,Viking Books,http://images.amazon.com/images/P/0670528951.0...,en,biography autobiography,"Based on two years of interviews and research,...",images/0670528951.01.THUMBZZZ.jpg
149563,0689818904,My Grandmother's Journey,John Cech,1998.0,Aladdin,http://images.amazon.com/images/P/0689818904.0...,en,fiction,A grandmother tells the story of her eventful ...,images/0689818904.01.THUMBZZZ.jpg
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket,http://images.amazon.com/images/P/067161746X.0...,en,humor,A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,nature,A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg


In [22]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import re
import pandas as pd
from tqdm import tqdm
from gensim.models import Word2Vec # numpy version 1.20.3 -> 1.24.2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

books = pd.read_csv('/opt/ml/data/books.csv')
users = pd.read_csv('/opt/ml/data/users.csv')
ratings = pd.read_csv('/opt/ml/data/train_ratings.csv')

In [23]:
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
books['category'] = books['category'].str.lower()

In [24]:
books.loc[books[books['category'] == 'juvenile fiction'].index, "category"] = 'fiction'
books.loc[books[books['category'] == 'juvenile nonfiction'].index, "category"] = 'nonfiction'

In [25]:
X, y = books[books['category'].notnull()]['book_title'], books[books['category'].notnull()]['category']


In [26]:
top_20_categories = y.value_counts().head(20).index.tolist()
top_20_categories

['fiction',
 'biography autobiography',
 'history',
 'religion',
 'nonfiction',
 'social science',
 'humor',
 'body mind spirit',
 'business economics',
 'cooking',
 'health fitness',
 'family relationships',
 'computers',
 'travel',
 'self help',
 'psychology',
 'poetry',
 'science',
 'art',
 'literary criticism']

In [27]:
# 상위 30개의 카테고리를 대상으로 함
top_20_categories = y.value_counts().head(20).index.tolist()
X = X[y.isin(top_20_categories)]
y = y[y.isin(top_20_categories)]

# 데이터셋을 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텍스트 데이터를 벡터화
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 로지스틱 회귀 모델 
clf = LogisticRegression(max_iter=2000)
clf.fit(X_train_vec, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = clf.predict(X_test_vec)

# 모델 평가
accuracy = accuracy_score(y_test, y_pred)
_classification_report = classification_report(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Classification Report: ", _classification_report) 


Accuracy:  0.7492071440494075
Classification Report:                           precision    recall  f1-score   support

                    art       0.65      0.20      0.31       120
biography autobiography       0.53      0.33      0.41       679
       body mind spirit       0.73      0.34      0.46       220
     business economics       0.58      0.39      0.46       194
              computers       0.84      0.46      0.59       140
                cooking       0.85      0.63      0.72       200
   family relationships       0.54      0.32      0.40       177
                fiction       0.77      0.98      0.87      7805
         health fitness       0.77      0.48      0.59       204
                history       0.52      0.28      0.36       401
                  humor       0.69      0.21      0.32       231
     literary criticism       0.60      0.12      0.20       103
             nonfiction       0.70      0.15      0.25       294
                 poetry       0.82 

In [28]:
_X = books[~books['category'].isin(top_20_categories)]['book_title']
_X_Vec = vectorizer.transform(_X)
_X_Vec

<89663x27249 sparse matrix of type '<class 'numpy.float64'>'
	with 438856 stored elements in Compressed Sparse Row format>

In [29]:
books

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,1940 1949,"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,medical,"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,fiction,A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,history,"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,humor,A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,nature,A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,fiction,These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [30]:
_y = clf.predict(_X_Vec)
_y

array(['fiction', 'fiction', 'biography autobiography', ..., 'fiction',
       'fiction', 'computers'], dtype=object)

In [31]:
books.loc[~books['category'].isin(top_20_categories), 'category'] = _y

In [32]:
books.to_csv('/opt/ml/data/preprocess_data/category_filled_books.csv')

In [33]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                1
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category                   0
summary                67227
img_path                   0
dtype: int64

In [34]:
books.drop(books.loc[books['book_author'].isna()].index, inplace=True)

In [35]:
value_count_df = pd.DataFrame(books['language'].value_counts())

In [20]:
# for small_lan in value_count_df[value_count_df['count'] < 100].index:
#     books.drop(books.loc[books['language'] == small_lan].index, inplace = True)
# books['language'].value_counts()

language
en    78823
de     1282
es     1017
fr      883
it      123
Name: count, dtype: int64

In [38]:
train_books = books[books['language'].notnull()]

In [39]:
X = train_books[['book_title', 'book_author', 'publisher']]
y = train_books['language']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train['book_title'] + ' ' + X_train['book_author'] + ' ' + X_train['publisher'])
X_test_vec = vectorizer.transform(X_test['book_title'] + ' ' + X_test['book_author'] + ' ' + X_test['publisher'])

clf = LogisticRegression(max_iter=2000)
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
_classification_report = classification_report(y_test, y_pred)

print('Accuracy: ', accuracy)
print("Classification Report : ", _classification_report)

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy:  0.9845770842188354
Classification Report :                precision    recall  f1-score   support

          ca       0.00      0.00      0.00         3
          da       0.00      0.00      0.00         8
          de       0.96      0.84      0.90       249
          el       0.00      0.00      0.00         1
          en       0.99      1.00      0.99     15761
          eo       0.00      0.00      0.00         1
          es       0.88      0.61      0.72       217
          fa       0.00      0.00      0.00         1
          fr       0.88      0.73      0.80       181
          it       1.00      0.33      0.50        27
          nl       1.00      0.10      0.18        10
          no       0.00      0.00      0.00         1
          pt       1.00      0.14      0.25         7
       zh-CN       0.00      0.00      0.00         2

    accuracy                           0.98     16469
   macro avg       0.48      0.27      0.31     16469
weighted avg       0.98  

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
_X = books[books['language'].isna()][['book_title', 'book_author', 'publisher']]
_X


,book_title,book_author,publisher
5,PLEADING GUILTY,Scott Turow,Audioworks
8,The Middle Stories,Sheila Heti,House of Anansi Press
9,Jane Doe,R. J. Kaiser,Mira Books
10,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,Health Communications
11,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,Brilliance Audio - Trade
...,...,...,...
149560,Town Like Alice,Nevil Shute,Ballantine Books
149561,LA Muerte Del Decano,Gonzalo Torrrente Ballester,Planeta Publishing Corporation
149564,The Motley Fool's What To Do with Your Money N...,David Gardner,Simon & Schuster Audio
149567,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,Lone Star Books


In [43]:
_X_Vec = vectorizer.transform(_X['book_title'] + ' ' + _X['book_author'] + ' ' + _X['publisher'])
_X_Vec

<67226x51736 sparse matrix of type '<class 'numpy.float64'>'
	with 594802 stored elements in Compressed Sparse Row format>

In [44]:
_y = clf.predict(_X_Vec)
_y

array(['en', 'en', 'en', ..., 'en', 'en', 'en'], dtype=object)

In [47]:
books.loc[books['language'].isna(), 'language'] = _y

In [48]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language                   0
category                   0
summary                67226
img_path                   0
dtype: int64

In [49]:
books.to_csv('/opt/ml/data/preprocess_data/category_language_filled_books.csv')

In [51]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language                   0
category                   0
summary                67226
img_path                   0
dtype: int64

In [53]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0])
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1])
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2])
users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

# city는 있는데 country 없는 경우 채우기
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in tqdm(modify_location, desc='(1/4) fill country'):
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

for location in tqdm(location_list, desc='(2/4) fill city'):
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

(1/4) fill country:  17%|█▋        | 360/2097 [00:09<00:46, 37.05it/s]/tmp/ipykernel_67543/690346601.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
(1/4) fill country:  61%|██████▏   | 1288/2097 [00:34<00:21, 37.58it/s]/tmp/ipykernel_67543/690346601.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
(2/4) fill city: 100%|██████████| 1948/1948 [00:20<00:00, 95.53it/s]


In [54]:
users

,user_id,location,age,location_city,location_state,location_country
0,8,"timmins, ontario, canada",NaN,timmins,ontario,canada
1,11400,"ottawa, ontario, canada",49.0,ottawa,ontario,canada
2,11676,"n/a, n/a, n/a",NaN,n/a,n/a,n/a
3,67544,"toronto, ontario, canada",30.0,toronto,ontario,canada
4,85526,"victoria, british columbia, canada",36.0,victoria,british columbia,canada
...,...,...,...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0,danville,california,usa
68088,278621,"victoria, delaware, canada",74.0,victoria,delaware,canada
68089,278636,"irvington, alabama, usa",NaN,irvington,alabama,usa
68090,278659,"vancouver, washington, usa",33.0,vancouver,british columbia,canada


In [55]:
ratings

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7
